# Value function iteration
## Uncertainty and the distribution of wealth


## The model

#### The agent's problem - recursive formulation

For a given level of assets $a$, the agent's value function is as follows:

$$ V(a_t,\theta_t) = \max_{c_t,a_{t+1}} u(c_t) + \beta\ \mathbb{E}[ V(a_{t+1},\theta_{t+1})]$$

subject to the budget constraint,

$$  q a_{t+1} + c_t \leq \theta_t + a_t,$$

where $\theta_t \in \Theta$ with $P(\theta_t = \theta) = \pi_\theta$

#### The agent's problem - recasted with one state variable

Our stochastic income process does not have any persistence; the realisation of $\theta_{t+1}$ is independent of the value of $\theta_{t}$. This means that we can re-cast the problem with only one state variable. There are a few ways to do this, but I'm going to choose the approach that keeps the envelope condition simple:

Let 
$$\hat{a}_t =  a_t + \theta_t.$$

We can re-write our agent's programme as follows

$$ \hat{V}(\hat{a}_t) = \max_{c_t,\hat{a}_{t+1}} u(c_t) + \beta\ \mathbb{E}[ \hat{V}(\hat{a}_{t+1})]$$

subject to the budget constraint,

$$  q (\hat{a}_{t+1}-\theta_{t+1})  \leq \hat{a}_t-c_t.$$


#### Envelope condition

$$ \hat{V}_{\hat{a}}(\hat{a}_t) = u'(c_t) $$

This equation will be used to calculate the optimal consumption choice (and as a conseqence the future asset holdings $a_{t+1}$) at each step of the iteration. 

#### Iterative solution method

For initial $a_t$ and initial value function $V_n$, the value of future asset choice $a_{t+1}$ is

$$V_{n+1}(\hat{a}_t) = \max_{c_t,\hat{a}_{t+1}} u(c_t) + \beta\ \mathbb{E}[ V_0(\hat{a}_{t+1})]$$

The ```v_update(v)``` function evaluates the above expression for $\forall a_t\in A,\theta_t\in\Theta$.

### Preamble

This notebook uses multi-threading. Check the output of `Threads.nthreads()` to ensure that you have multi-threading enabled. If the function returns `1`, [update your multi-threading environment variable](https://docs.julialang.org/en/v1/manual/multi-threading/). 

In [ ]:
Threads.nthreads()

In [ ]:
# using Pkg;
# Pkg.add.(["StatsBase","SpecialFunctions"]);

In [ ]:
using Interpolations;
using PGFPlots
using SpecialFunctions;
using StatsBase;
using Optim;
using BenchmarkTools;

In [ ]:
include("functions/distribution_functions.jl");
include("functions/plotting_functions.jl");

# Model type

In [ ]:
mutable struct OptimalPolicy
    value::Vector{Float64}
    assets::Vector{Any}    
    consumption::Vector{Float64}
end

In [ ]:
mutable struct SimulatedData
    income::Vector{Float64}
    assets::Vector{Float64}    
    consumption::Vector{Float64}
end

In [ ]:
mutable struct Distribution
    pdf_grid::Vector{Float64}
    pdf_uniform::Vector{Float64}    
    cdf::Vector{Float64}
    lorenz_consumption::Vector{Float64}    
    lorenz_assets::Vector{Float64}            
    gini_income::Float64    
    gini_consumption::Float64      
    gini_assets::Float64                   
end

In [ ]:
mutable struct Grid
    gridpoints::Vector # grid values
    gridmin::Float64   # minimum value
    gridmax::Float64   # maximum value
    ngrid::Int         # number of gridpoints
end

In [ ]:
mutable struct Model
    parameters::Dict
    grid::Union{Grid,Missing}
    optimal_policy::Union{OptimalPolicy,Missing}
    simulated_data::Union{SimulatedData,Missing}
    distribution::Union{Distribution,Missing}
end

#### Convenience constructor

A convenient way to construct new models

In [ ]:
Model(parameters::Dict) = Model(parameters::Dict,missing,missing,missing,missing)

## Parameters

_Preferences_

In [ ]:
β = 0.968; # time preference
σ = 2;     # CRRA

u(c) = c > 0 ? (c^(1-σ))/(1-σ) : -Inf; # felicity

_Production_

We generate income draws from a [zeta distribution](https://en.wikipedia.org/wiki/Zeta_distribution). This is the discrete equivalent of a Pareto distribution.

In [ ]:
n_θ = 10;                                # Income gridpoints
θ   = range(1.0,stop=n_θ,length=n_θ);    # Income grid
s_θ = 1.01;                              # Distribution parameter (higher is more unequal)
π_θ = [k^(-s_θ)/zeta(s_θ) for k in θ];   # Probability mass function
π_θ = π_θ./sum(π_θ);                     # Normalise sum(pmf) = 1

In [ ]:
π_θ

_Prices_

In [ ]:
q = 1/1.03;

In [ ]:
m = Model(
        Dict(:β   => β,
             :σ   => σ,
             :u   => u,
             :n_θ => n_θ,
             :θ   => θ,
             :π_θ => π_θ,
             :q   => q
        )
)

#### Initialise asset grid

The lifetime wealth of an agent is the sum of asset holdings and the present value of future income streams, we can calculate a lower bound on lifetime wealth under the assumption that the agent earns the minimum endowment in all future periods

$$\underline{w}(a) = \hat{a} + \sum_{t=0}^\infty q^t \underline{\theta}.$$

This value can approach zero while still supporting positive consumption in all future states. So, we construct an exponentially-spaced grid over $\underline{w}$, then from that construct a grid over $a$.

In [ ]:
w_min  = 2.0;
w_max  = 300.0;
n_grid = 50;
w_grid = exp.(range(log(w_min),stop=log(w_max),length=n_grid))

a_grid = w_grid .- (1/(1-q))*θ[1];
a_min  = minimum(a_grid)
a_max  = maximum(a_grid)

m.grid = Grid(a_grid,a_min,a_max,n_grid)

#### Generate initial guess

In [ ]:
c0 = (1-β).*w_grid;
v0 = (1/(1-β)).*u.(c0);

push!(m.parameters,:c0 => c0);
push!(m.parameters,:v0 => v0);

In [ ]:
function v_update(v,model::Model)
    
    a_grid = model.grid.gridpoints
    a_min  = model.grid.gridmin 
    a_max  = model.grid.gridmax 
    n_grid = model.grid.ngrid     
    β      = model.parameters[:β]
    σ      = model.parameters[:σ]
    q      = model.parameters[:q]    
    θ      = model.parameters[:θ]
    n_θ    = model.parameters[:n_θ]    
    π_θ    = model.parameters[:π_θ]        
    u      = model.parameters[:u]            
    
    v_itp   = interpolate(a_grid,v,SteffenMonotonicInterpolation())
    
    v_a(a) = Interpolations.gradient(v_itp, a)[1]
    
    cstar = Vector{Float64}(undef,n_grid)
    astar = Vector{Vector{Float64}}(undef,n_grid)
    vstar = Vector{Float64}(undef,n_grid)    
    
    Threads.@threads for i in 1:n_grid
    
        cstar[i] = min(max(v_a(a_grid[i])^(-1/σ),a_grid[i]-q*(a_max-θ[n_θ])),a_grid[i]-q*(a_min-θ[1])) 
        astar[i] = (a_grid[i] - cstar[i])/q .+ θ
        vstar[i] = u(cstar[i]) .+ β* π_θ'*v_itp.(astar[i])
    
    end    
        
    return (vstar,astar,cstar)
end

In [ ]:
(vs,as,cs) = v_update(v0,m);

In [ ]:
as[1]

To solve the model, we iterate over value functions as follows:

$$ V_1(a) = \max_{a'\in A} u(y + Ra- a') + \beta\  V_0(a')$$

In [ ]:
function solve!(model;max_iter=1000,tol_d=1e-2,verbose=false)

    a_grid = model.grid.gridpoints
    v0     = model.parameters[:v0]
    
    v      = deepcopy(v0);
    vprime = deepcopy(v);
    aprime = Array{Float64,1}(a_grid);
    cstar  = zeros(size(a_grid));    
    d      = 1000;
    i      = 1;
    
    while i <= max_iter && d > tol_d
        (vprime,aprime,cstar) = v_update(v,model) # vprime_int(v) 
        i     += 1
        d      = (vprime .- v).^2 |> sum |> sqrt
        v      = deepcopy(vprime)
    end

    if verbose
        println("iteration = $i")
        println("distance  = $(round(d,digits=6))")
    end
    
    model.optimal_policy = OptimalPolicy(v,aprime,cstar)    

end;

In [ ]:
@time solve!(m;max_iter=1000,tol_d=1e-5);

### Plotting the results

In [ ]:
plot_optimalpolicy(m)

# A typical path

In [ ]:
function generate_path!(model;n_periods=100,burn=100,a_init=0.0)

    a_grid = model.grid.gridpoints   
    θ      = model.parameters[:θ]
    n_θ    = model.parameters[:n_θ]    
    π_θ    = model.parameters[:π_θ]        
    vstar  = model.optimal_policy.value
    astar  = model.optimal_policy.assets    
    cstar  = model.optimal_policy.consumption
    
    θ_index_path = sample( 1:n_θ, ProbabilityWeights(π_θ), n_periods+burn)
    a_path = zeros(n_periods+burn)
    c_path = zeros(n_periods+burn)
    a_path[1] = a_init

    int_astar = interpolate.([(collect(a_grid),)],[[a[i] for a in astar] for i in 1:n_θ],[Gridded(Linear())])
    int_cstar = interpolate((collect(a_grid),),cstar,Gridded(Linear()))

    c_path[1] = int_cstar(a_init)
    
    for τ = 2:n_periods + burn
        a_path[τ] = int_astar[θ_index_path[τ]](a_path[τ-1])
        c_path[τ] = int_cstar(a_path[τ])        
    end
    
    θ_path = [θ[i] for i in θ_index_path]
    
    model.simulated_data = SimulatedData(θ_path[burn+1:n_periods+burn],
                                         a_path[burn+1:n_periods+burn],
                                         c_path[burn+1:n_periods+burn])  
    
end

In [ ]:
generate_path!(m)

In [ ]:
plot_simulateddata(m)

## Stationary distribution

We solve for the stationary distribution by starting with an initial guess distribution, then simulating the model until the distribution converges.

In [ ]:
# Initial guess distribution
ϕ_0 = ones(n_grid)./n_grid;
push!(m.parameters,:ϕ_0 => ϕ_0);

One function that will help us a lot is an interpolant that quickly returns the index number associated with a specific gridpoint. We create that below and add it to the parameters of our model:

In [ ]:
int_index(model) = interpolate((collect(model.grid.gridpoints),),collect(1:size(model.grid.gridpoints)[1]),Gridded(Linear()))
push!(m.parameters,:int_index => int_index(m));

In [ ]:
function ϕ_update(ϕ,model)

    astar  = model.optimal_policy.assets
    n_grid = model.grid.ngrid
    π_θ    = model.parameters[:π_θ]
    n_θ    = model.parameters[:n_θ]  
    
    int_index = model.parameters[:int_index]    
    
    ϕ_prime = zeros(n_grid)
    index   = Float64(0.0)
    inddown = Int(0)
    indup   = Int(0)

    for i in 1:n_grid, j in 1:n_θ
           index   = int_index(astar[i][j]) 
           inddown = index |> floor |> Int
           indup   = index |> ceil |> Int
           ϕ_prime[inddown] = ϕ_prime[inddown] + π_θ[j]*ϕ[i]*(indup - index)
           ϕ_prime[indup]   = ϕ_prime[indup]   + π_θ[j]*ϕ[i]*(index - inddown)
    end
    
    return ϕ_prime./sum(ϕ_prime)

end

In [ ]:
@btime ϕ_update(ϕ_0,m);

In [ ]:
function solve_distribution!(model;max_iter=10000,tol_d=1e-6)
    
    int_index(model) = interpolate((collect(model.grid.gridpoints),),collect(1:size(model.grid.gridpoints)[1]),Gridded(Linear()))
    push!(model.parameters,:int_index => int_index(model));
    
    ϕ_0    = model.parameters[:ϕ_0]  
    astar  = model.optimal_policy.assets
    cstar  = model.optimal_policy.consumption
    a_grid = model.grid.gridpoints
    n_grid = size(model.grid.gridpoints)[1]    
    π_θ    = model.parameters[:π_θ]
    θ      = model.parameters[:θ]    
    a_trunc= [maximum([a,0.0]) for a in a_grid]
    
    ϕ       = deepcopy(ϕ_0);
    ϕ_prime = deepcopy(ϕ_0);
    
    d      = 1000;
    i      = 1;
    
    while i <= max_iter && d > tol_d
        ϕ_prime = ϕ_update(ϕ,model) # vprime_int(v) 
        i      += 1
        d       = (ϕ_prime .- ϕ).^2 |> sum |> sqrt
        ϕ       = deepcopy(ϕ_prime)
    end

    cdf    = [sum(ϕ[1:i]) for i in 1:n_grid]
    
    model.distribution = Distribution(
                            ϕ,
                            genpdf(cdf,model.grid.gridpoints),
                            cdf,
                            Lorenz(ϕ,cstar),
                            Lorenz(ϕ,a_trunc),
                            gini(π_θ,θ), 
                            gini(ϕ,cstar),
                            gini(ϕ,a_trunc)
                            )
                
    
end;

In [ ]:
@btime solve_distribution!(m;max_iter=100000,tol_d=1e-6);

#### Plotting the results

In [ ]:
plot_distributions(m)

# Parameter estimation (complete for your problem set)

We start by creating a function that takes a vector of parameters as inputs and outputs a solved model.

`parameters` is a vector containing $[\beta,\sigma,s_{\theta}]$

In [ ]:
function generate_moments(parameters::Vector{Float64})
    
    s_θ = parameters[3];                     # Distribution parameter (higher is more unequal)
    π_θ = [k^(-s_θ)/zeta(s_θ) for k in θ];   # Probability mass function
    π_θ = π_θ./sum(π_θ);                     # Normalise sum(pmf) = 1
    
    m = Model(
        Dict(:β   => parameters[1],
             :σ   => parameters[2],
             :u   => u,
             :n_θ => n_θ,
             :θ   => θ,
             :π_θ => π_θ,
             :q   => q
        )
    )
    
    w_min  = 2.0;
    w_max  = 300.0;
    n_grid = 50;
    w_grid = exp.(range(log(w_min),stop=log(w_max),length=n_grid))

    a_grid = w_grid .- (1/(1-q))*θ[1];
    a_min  = minimum(a_grid)
    a_max  = maximum(a_grid)

    m.grid = Grid(a_grid,a_min,a_max,n_grid)
    
    c0 = (1-β).*w_grid;
    v0 = (1/(1-β)).*u.(c0);

    push!(m.parameters,:c0 => c0);
    push!(m.parameters,:v0 => v0);
    
    solve!(m;max_iter=1000,tol_d=1e-5);
    
    ϕ_0 = ones(n_grid)./n_grid;
    push!(m.parameters,:ϕ_0 => ϕ_0);
    
    solve_distribution!(m;max_iter=100000,tol_d=1e-6);
    
    return m
end

In [ ]:
generate_moments([0.968,2.2,1.02]).distribution.gini_consumption

In [ ]:
m.distribution.gini_consumption